In [2]:
# Import Libraires
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocessing

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',	'USE_CASE',	'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'])

In [8]:
# Split our preprocessed data into our features and target arrays
y = application_dummies['IS_SUCCESSFUL']
X = application_dummies.drop(['IS_SUCCESSFUL'], axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
print(X_train.shape)

(25724, 43)


# PyTorch Model

In [47]:
import torch.nn as nn

# Define the model class
class NeuralNetwork(nn.Module):
  def __init__(self, input_features):
    super(NeuralNetwork, self).__init__()
    self.layer1 = nn.Linear(input_features, 300) # First hidden layer
    self.layer2 = nn.Linear(300, 150)        # Second hidden layer
    self.output = nn.Linear(150, 1)         # Output layer
    self.activation = nn.ReLU()            # ReLU activation function
    self.sigmoid = nn.Sigmoid()             # Sigmoid activation for output

  def forward(self, x):
    x = self.activation(self.layer1(x))    # Apply first layer activation
    x = self.activation(self.layer2(x))    # Apply second layer activation
    x = self.sigmoid(self.output(x))       # Apply output layer and sigmoid activation
    return x

# Instantiate the model
input_features = 43
model = NeuralNetwork(input_features)

# Print model
print(model)

NeuralNetwork(
  (layer1): Linear(in_features=43, out_features=300, bias=True)
  (layer2): Linear(in_features=300, out_features=150, bias=True)
  (output): Linear(in_features=150, out_features=1, bias=True)
  (activation): ReLU()
  (sigmoid): Sigmoid()
)


In [48]:
# Compile the model
import torch.optim as optim
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer

In [51]:
# Training the model
epochs = 100
for epoch in range(epochs):
  model.train()

  # Convert training data to tensors
  inputs = torch.FloatTensor(X_train_scaled)
  labels = torch.FloatTensor(y_train.values)

  # Zero the gradients
  optimizer.zero_grad()

  # Forward pass
  outputs = model(inputs)

  # Compute the loss
  loss = criterion(outputs, labels.view(-1, 1))

  # Backward pass and optimization
  loss.backward()
  optimizer.step()

  # Calculate accuracy
  with torch.no_grad():
    predicted = (outputs > 0.5).FloatTensor()
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)

  # Print loss and accuracy every 5 epochs
  if (epoch+1) % 5 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

AttributeError: 'Tensor' object has no attribute 'FloatTensor'

AttributeError: module 'torch.nn' has no attribute 'evaluate'